## Read in data

In [1]:
import pandas as pd
import numpy as np

In [2]:
oo = pd.read_csv('../data/online_retail_small.csv')

In [3]:
d1 = oo.sample(n=4)

In [4]:
d2 = oo.sample(n=4)

## Column splitter

In [5]:
d2 = oo.sample(n=4)

In [6]:
d2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2665,536592,22169,FAMILY ALBUM WHITE PICTURE FRAME,1,1/12/2010 17:06,16.98,NaN,United Kingdom
4337,536763,21733,RED HANGING HEART T-LIGHT HOLDER,6,2/12/2010 14:42,2.95,17685.0,United Kingdom
1609,536544,22569,FELTCRAFT CUSHION BUTTERFLY,1,1/12/2010 14:32,7.62,NaN,United Kingdom
1906,536544,21121,SET/10 RED POLKADOT PARTY CANDLES,7,1/12/2010 14:32,2.51,NaN,United Kingdom


In [7]:
d2.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [8]:
def column_splitter(df, col_name, regex_string="(?=-)"):
    org_col_index = df.columns
    split_id = np.argwhere(org_col_index == col_name).reshape(1)[0]
    
    # split the original df into three sections: 
    # to_keep, col_to_split and cols_to_join_back
    to_keep = df.iloc[:, :split_id].copy(deep=True)
    to_split = df[[col_name]].copy(deep=True)
    to_join = df.iloc[:, (split_id+1):].copy(deep=True)
    cols_to_add = np.hstack((org_col_index[(split_id+1):], ''))
    to_join[''] = pd.Series([np.NaN]*to_join.shape[0])
    
    #split the column:
    to_split = to_split[col_name].str.split(regex_string, n=1, expand=True)
    to_split.columns = [col_name, cols_to_add[0]]
    
    # join the split column back first
    #to_keep = pd.concat([to_keep, to_split], axis=1)
    to_keep = to_keep.join(to_split)
    na_boolean = to_keep[cols_to_add[0]].isna()
    to_keep = to_keep.combine_first(to_join[[cols_to_add[0]]])
    #breakpoint()
    
    for i in np.arange(1,len(cols_to_add)):
        #print(i)
        new_col = to_join.iloc[:, i].copy(deep=True)
        new_col[~na_boolean] = to_join.iloc[:, i-1][~na_boolean]
        to_keep = pd.concat([to_keep, new_col], axis=1)
        #print(cols_to_add[i])
    #print(cols_to_add)
    
    return to_keep[np.hstack((org_col_index, ''))]

In [9]:
column_splitter(d2, 'Description')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,
2665,536592,22169,FAMILY ALBUM WHITE PICTURE FRAME,1,1/12/2010 17:06,16.98,NaN,United Kingdom,NaN
4337,536763,21733,RED HANGING HEART T,-LIGHT HOLDER,6.0,2/12/2010 14:42,2.95,17685.0,United Kingdom
1609,536544,22569,FELTCRAFT CUSHION BUTTERFLY,1,1/12/2010 14:32,7.62,NaN,United Kingdom,NaN
1906,536544,21121,SET/10 RED POLKADOT PARTY CANDLES,7,1/12/2010 14:32,2.51,NaN,United Kingdom,NaN


## Column joiner

In [10]:
telco = pd.read_csv("../data/Telco-Customer-Churn.csv")

In [132]:
telco_sub = telco.sample(n=10)
telco_sub

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6279,4012-ZTHBR,Female,0,Yes,Yes,21,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,44.95,926.25,No
4345,4302-ZYFEL,Male,0,Yes,Yes,69,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.55,1403.10,No
3449,5473-KHBPS,Female,0,Yes,Yes,52,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,No,One year,Yes,Credit card (automatic),95.70,4976.15,No
2081,5934-TSSAU,Female,0,Yes,Yes,23,Yes,No,Fiber optic,No,...,Yes,No,No,No,Month-to-month,Yes,Credit card (automatic),75.30,1702.90,No
6684,0305-SQECB,Female,0,No,Yes,11,No,No phone service,DSL,Yes,...,No,Yes,No,No,One year,Yes,Mailed check,36.05,402.60,No
2394,8231-BSWXX,Male,0,No,No,71,Yes,Yes,Fiber optic,No,...,No,No,No,No,One year,Yes,Credit card (automatic),79.05,5552.50,No
1511,9972-EWRJS,Female,0,Yes,Yes,67,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.90,No
1584,9906-NHHVC,Female,1,No,No,65,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),53.50,3517.90,No
867,2732-ISEZX,Female,0,No,No,5,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,20.50,104.30,No
779,0322-CHQRU,Male,0,No,No,2,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,45.35,89.50,Yes


In [133]:
c1 = join_fn(telco_sub.MultipleLines, telco_sub.InternetService)

In [63]:
def join_fn(v1, v2):
    return (v1 == 'No phone service') & (v2 == 'DSL')

In [137]:
def column_joiner(df, col_name1, col_name2, join_criteria):
    # join_index should be a boolean index that indicates which rows to join.
    # use *row_idx* when we implement the stainer.
    
    org_col_index = df.columns
    split_id = np.argwhere(org_col_index == col_name2).reshape(1)[0]
    to_keep = df.loc[:, :col_name1].copy(deep=True)
    join_series = df[col_name2].copy(deep=True)
    to_join = df.iloc[:, split_id:].copy(deep=True)
    
    # breakpoint()
    # modify the column to join, and paste with to_keep
    join_series[~join_criteria] = ''
    to_keep[col_name1] = to_keep[col_name1] + join_series
    
    for i in np.arange(split_id, df.shape[1]):
        # print(i)
        new_col = df.iloc[:, i].copy(deep=True)
        if i < (df.shape[1] - 1):
            new_col[join_criteria] = df.iloc[:, i+1][join_criteria]
        else:
            new_col[join_criteria] = np.NaN
        to_keep = pd.concat([to_keep, new_col], axis=1)
        #breakpoint()

    return to_keep

In [138]:
out = column_joiner(telco_sub, 'MultipleLines', 'InternetService', c1)

In [142]:
telco_sub.loc[:, 'MultipleLines':]

,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6279,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,44.95,926.25,No
4345,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.55,1403.10,No
3449,Yes,Fiber optic,No,Yes,Yes,No,Yes,No,One year,Yes,Credit card (automatic),95.70,4976.15,No
2081,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Credit card (automatic),75.30,1702.90,No
6684,No phone service,DSL,Yes,No,No,Yes,No,No,One year,Yes,Mailed check,36.05,402.60,No
2394,Yes,Fiber optic,No,Yes,No,No,No,No,One year,Yes,Credit card (automatic),79.05,5552.50,No
1511,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.90,No
1584,No phone service,DSL,No,Yes,Yes,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),53.50,3517.90,No
867,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,20.50,104.30,No
779,No,DSL,No,No,No,No,No,No,Month-to-month,No,Electronic check,45.35,89.50,Yes


## Resample residuals

In [123]:
out['MultipleLines'] + out['OnlineBackup']

3900    NoNo internet service
4419    NoNo internet service
6752                      NaN
941                       NaN
5763                     NoNo
222     NoNo internet service
5028                    NoYes
5974    NoNo internet service
5620                   YesYes
380                     NoYes
dtype: object

In [117]:
out.shape

(10, 21)

In [46]:
tmp[['Description', 'InvoiceNo']]

,Description,InvoiceNo
3207,GLASS STAR FROSTED T,536609
1648,ALARM CLOCK BAKELIKE PINK,536544
4164,EDWARDIAN PARASOL BLACK,536750
4482,SET 7 BABUSHKA NESTING BOXES,536784


In [16]:
tmp

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,
4719,536798,21899,KEY FOB,", GARAGE DESIGN",2/12/2010 15:55,0.65,17838.00,United Kingdom,NaN
747,536446,20777,CHRYSANTHEMUM NOTEBOOK,None,2.0,1/12/2010 12:15,1.65,15983.0,United Kingdom
2132,536561,22274,FELTCRAFT DOLL EMILY,None,6.0,1/12/2010 15:06,2.95,12921.0,United Kingdom
3027,536592,90166,PINK & WHITE ROSEBUD RING,None,1.0,1/12/2010 17:06,4.24,NaN,United Kingdom


In [ ]:
pdb.runcall(column_splitter, d2, 'Description')

In [77]:
tmp.iloc[:, 5]

1097    NaN
4718    NaN
3193    NaN
4921    NaN
Name: , dtype: object

In [10]:
pd.DataFrame.combine_first?

Signature: pd.DataFrame.combine_first(self, other: 'DataFrame') -> 'DataFrame'
Docstring:
Update null elements with value in the same location in `other`.

Combine two DataFrame objects by filling null values in one DataFrame
with non-null values from other DataFrame. The row and column indexes
of the resulting DataFrame will be the union of the two.

Parameters
----------
other : DataFrame
    Provided DataFrame to use to fill null values.

Returns
-------
DataFrame

See Also
--------
DataFrame.combine : Perform series-wise operation on two DataFrames
    using a given function.

Examples
--------
>>> df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
>>> df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
>>> df1.combine_first(df2)
     A    B
0  1.0  3.0
1  0.0  4.0

Null values still persist if the location of that null value
does not exist in `other`

>>> df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
>>> df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
>>> df1.comb

In [30]:
id = pd.Series([True, False, False, True])

In [31]:
id.index = qty_series.index

In [32]:
qty_series[id] = d2.Description[id]

In [33]:
qty_series

3159    FULL ENGLISH BREAKFAST PLATE
3348                            12.0
4596                             2.0
4405     ZINC METAL HEART DECORATION
Name: Quantity, dtype: object

In [24]:
d2a = d2.Description.str.split('-',n=1, expand=True)

In [25]:
d2a[[1]]

,1
4608,None
1675,None
4451,None
1762,LIGHT HLDR


In [26]:
d2[["Quantity"]]

,Quantity
4608,1
1675,1
4451,108
1762,4


In [28]:
d2.combine_first?

Signature: d2.combine_first(other: 'DataFrame') -> 'DataFrame'
Docstring:
Update null elements with value in the same location in `other`.

Combine two DataFrame objects by filling null values in one DataFrame
with non-null values from other DataFrame. The row and column indexes
of the resulting DataFrame will be the union of the two.

Parameters
----------
other : DataFrame
    Provided DataFrame to use to fill null values.

Returns
-------
DataFrame

See Also
--------
DataFrame.combine : Perform series-wise operation on two DataFrames
    using a given function.

Examples
--------
>>> df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
>>> df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
>>> df1.combine_first(df2)
     A    B
0  1.0  3.0
1  0.0  4.0

Null values still persist if the location of that null value
does not exist in `other`

>>> df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
>>> df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
>>> df1.combine_first(df2)
 

In [27]:
pd.concat([d2a[[1]],d2[["Quantity"]]], axis=1)

,1,Quantity
4608,None,1
1675,None,1
4451,None,108
1762,LIGHT HLDR,4


In [65]:
oo.Description.str.contains("[^\w\s]", regex=True).isna().sum()

12

In [68]:
oo2 = oo[~oo.Description.isna()]

In [73]:
tmp = oo2[oo2.Description.str.contains("[^\w\s]", regex=True)].Description

In [77]:
tmp.str.split("(?=-)",n=1, expand=True)

,0,1
0,WHITE HANGING HEART T,-LIGHT HOLDER
4,RED WOOLLY HOTTIE WHITE HEART.,None
6,GLASS STAR FROSTED T,-LIGHT HOLDER
10,POPPY'S PLAYHOUSE BEDROOM,None
11,POPPY'S PLAYHOUSE KITCHEN,None
...,...,...
4969,SET OF 6 T,-LIGHTS SNOWMEN
4982,DINOSAUR PARTY BAG + STICKER SET,None
4986,EUCALYPTUS & PINECONE WREATH,None
4988,"SWISS ROLL TOWEL, CHOCOLATE SPOTS",None
